In [1]:
!pip install -U sentence-transformers
!pip install pandas
!pip install accelerate==0.32.0
!pip install torch
!pip install summac
!pip install nltk
!pip install --upgrade evaluate
!pip install lm_eval
!pip install huggingface_hub=0.13.3
!pip install bert_score
!pip install peft==0.10.0
!pip install -U transformers

  Using cached transformers-4.52.4-py3-none-any.whl.metadata (38 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-macosx_11_0_arm64.whl.metadata (6.8 kB)
Using cached transformers-4.52.4-py3-none-any.whl (10.5 MB)
Using cached tokenizers-0.21.1-cp39-abi3-macosx_11_0_arm64.whl (2.7 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
summac 0.0.4 requires huggingface-hub<=0.17.0, but you have huggingface-hub 0.33.0 which is incompatible.
  Using cached huggingface_hub-0.17.0-py3-none-any.whl.metadata (13 kB)
INFO: pip is 

In [3]:
import pandas as pd
from tqdm import tqdm

In [ ]:
filepath = './articles_simplified_4.jsonl'
articles = pd.read_json(path_or_buf=filepath, lines=True)
texts = articles['text']
simplifieds = articles['simplified']
simplifieds2 = articles['simplified2']
simplifieds3 = articles['simplified3']

In [4]:
filepath = './output_with_sentence_counts.json'
articles = pd.read_json(path_or_buf=filepath)
texts = articles['Article']
simplifieds = articles['GPT_Result']

new_filepath = './sample.json'
new_articles = pd.read_json(path_or_buf=new_filepath)
level1 = list(new_articles[new_articles['Level'] == 1]["GPT_Result"])
level2 = list(new_articles[new_articles['Level'] == 2]["GPT_Result"])
level3 = list(new_articles[new_articles['Level'] == 3]["GPT_Result"])
level4 = list(new_articles[new_articles['Level'] == 4]["GPT_Result"])
level5 = list(new_articles[new_articles['Level'] == 5]["GPT_Result"])
level6 = list(new_articles[new_articles['Level'] == 6]["GPT_Result"])
level7 = list(new_articles[new_articles['Level'] == 7]["GPT_Result"])
level8 = list(new_articles[new_articles['Level'] == 8]["GPT_Result"])
level9 = list(new_articles[new_articles['Level'] == 9]["GPT_Result"])

In [5]:
import nltk

nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/ytzka14/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [24]:
from sentence_transformers import SentenceTransformer
import sklearn.metrics.pairwise as pairwise


model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
# csv_str = "article_id, sim1, sim2, sim3\n"
csv_str = "article_id, sim0, simlev1, simlev2, simlev3, simlev4, simlev5, simlev6, simlev7, simlev8, simlev9\n"

print(len(texts), len(simplifieds), len(level1), len(level2), len(level3), len(level4), len(level5), len(level6), len(level7), len(level8), len(level9))
assert len(texts) == len(simplifieds) == len(level1) == len(level2) == len(level3) == len(level4) == len(level5) == len(level6) == len(level7) == len(level8) == len(level9)

for i in tqdm(range(len(texts))):
  # sentences = [texts[i], simplifieds[i], simplifieds2[i], simplifieds3[i]]
  sentences = [texts[i], simplifieds[i], level1[i], level2[i], level3[i], level4[i], level5[i], level6[i], level7[i], level8[i], level9[i]]
  embeddings = model.encode(sentences)
  similarities = pairwise.cosine_similarity(embeddings)
  sim0 = similarities[0][1]
  simlev1 = similarities[0][2]
  simlev2 = similarities[0][3]
  simlev3 = similarities[0][4]
  simlev4 = similarities[0][5]
  simlev5 = similarities[0][6]
  simlev6 = similarities[0][7]
  simlev7 = similarities[0][8]
  simlev8 = similarities[0][9]
  simlev9 = similarities[0][10]
  # sim2 = similarities[0][2]
  # sim3 = similarities[0][3]
  # csv_str += f"{i}, {sim1}, {sim2}, {sim3}\n"
  csv_str += f"{i}, {sim0}, {simlev1}, {simlev2}, {simlev3}, {simlev4}, {simlev5}, {simlev6}, {simlev7}, {simlev8}, {simlev9}\n"

with open('new_all-MiniLM-L6-v2.csv', 'w') as f:
  f.write(csv_str)



90 90 90 90 90 90 90 90 90 90 90


100%|██████████| 90/90 [00:16<00:00,  5.45it/s]


In [25]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("biu-nlp/superpal")
model = AutoModelForSequenceClassification.from_pretrained("biu-nlp/superpal")

def get_similarity(text1, text2):
    # For sequence classification on sentence pairs,
    # pass the two texts as separate arguments.
    inputs = tokenizer(text1, text2, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    # Apply softmax over the logits to get probabilities for each class.
    probs = outputs.logits.softmax(dim=1)
    # Assuming that index 1 corresponds to the "similar" (paraphrase) class.
    similarity_score = probs[0][1].item()
    return similarity_score

# csv_str = "article_id, sim1, sim2, sim3\n"
csv_str = "article_id, sim0, simlev1, simlev2, simlev3, simlev4, simlev5, simlev6, simlev7, simlev8, simlev9\n"

for i in tqdm(range(articles.shape[0])):
    sim0 = get_similarity(texts[i], simplifieds[i])
    simlev1 = get_similarity(texts[i], level1[i])
    simlev2 = get_similarity(texts[i], level2[i])
    simlev3 = get_similarity(texts[i], level3[i])
    simlev4 = get_similarity(texts[i], level4[i])
    simlev5 = get_similarity(texts[i], level5[i])
    simlev6 = get_similarity(texts[i], level6[i])
    simlev7 = get_similarity(texts[i], level7[i])
    simlev8 = get_similarity(texts[i], level8[i])
    simlev9 = get_similarity(texts[i], level9[i])
    # csv_str += f"{i}, {sim1}, {sim2}, {sim3}\n"
    csv_str += f"{i}, {sim0}, {simlev1}, {simlev2}, {simlev3}, {simlev4}, {simlev5}, {simlev6}, {simlev7}, {simlev8}, {simlev9}\n"

with open('new_superpal.csv', 'w') as f:
    f.write(csv_str)

100%|██████████| 90/90 [37:38<00:00, 25.10s/it]   


In [26]:
import evaluate

rouge = evaluate.load('rouge')
# csv_str = "article_id, sim1, sim2, sim3\n"
csv_str = "article_id, sim0, simlev1, simlev2, simlev3, simlev4, simlev5, simlev6, simlev7, simlev8, simlev9\n"

for i in tqdm(range(articles.shape[0])):
  predictions = [simplifieds[i], level1[i], level2[i], level3[i], level4[i], level5[i], level6[i], level7[i], level8[i], level9[i]]
  references = [texts[i], texts[i], texts[i], texts[i], texts[i], texts[i], texts[i], texts[i], texts[i], texts[i]]
  results = rouge.compute(predictions=predictions, references=references, use_aggregator=False, rouge_types=['rougeL'])
  rougeL = results['rougeL']
  csv_str += f"{i}, {rougeL[0]}, {rougeL[1]}, {rougeL[2]}, {rougeL[3]}, {rougeL[4]}, {rougeL[5]}, {rougeL[6]}, {rougeL[7]}, {rougeL[8]}, {rougeL[9]}\n"

with open('new_rougeL.csv', 'w') as f:
  f.write(csv_str)


100%|██████████| 90/90 [01:06<00:00,  1.36it/s]


In [6]:
import evaluate

bertscore = evaluate.load('bertscore')
# csv_str = "article_id, sim1, sim2, sim3\n"
csv_str = "article_id, sim0, simlev1, simlev2, simlev3, simlev4, simlev5, simlev6, simlev7, simlev8, simlev9\n"

for i in tqdm(range(articles.shape[0])):
  predictions = [simplifieds[i], level1[i], level2[i], level3[i], level4[i], level5[i], level6[i], level7[i], level8[i], level9[i]]
  references = [texts[i], texts[i], texts[i], texts[i], texts[i], texts[i], texts[i], texts[i], texts[i], texts[i]]
  results = bertscore.compute(predictions=predictions, references=references, lang="en")
  f1score = results['f1']
  csv_str += f"{i}, {f1score[0]}, {f1score[1]}, {f1score[2]}, {f1score[3]}, {f1score[4]}, {f1score[5]}, {f1score[6]}, {f1score[7]}, {f1score[8]}, {f1score[9]}\n"

with open('new_bertscore.csv', 'w') as f:
  f.write(csv_str)

  0%|          | 0/90 [00:00<?, ?it/s]Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 90/90 [15:45<00:00, 10.51s/it]


In [ ]:
# from summac.model_summac import SummaCZS, SummaCConv

# model_zs = SummaCZS(granularity='sentence', model_name='vitc', device='cpu')
# model_conv = SummaCConv(models=['vitc'], bins='percentile', granularity='sentence', nli_labels='e', device='cpu', start_file='default', agg='mean')

# zs_csv_str = "article_id, sim1, sim2, sim3\n"
# conv_csv_str = "article_id, sim1, sim2, sim3\n"

# for i in range(articles.shape[0]):
#   score_zs = model_zs.score([texts[i], texts[i], texts[i]], [simplifieds[i], simplifieds2[i], simplifieds3[i]])
#   score_conv = model_conv.score([texts[i], texts[i], texts[i]], [simplifieds[i], simplifieds2[i], simplifieds3[i]])

#   score_zs = score_zs["scores"]
#   score_conv = score_conv["scores"]
  
#   zs_csv_str += f"{i}, {score_zs[0]}, {score_zs[1]}, {score_zs[2]}\n"
#   conv_csv_str += f"{i}, {score_conv[0]}, {score_conv[1]}, {score_conv[2]}\n"

# with open('SummaCZS.csv', 'w') as f:
#   f.write(zs_csv_str)
#   f.close()

# with open('SummaCConv.csv', 'w') as f:
#   f.write(conv_csv_str)
#   f.close()